In [ ]:
"""
Count pills with YOLOv8n (smallest) + circle-shape filter + HoughCircles fallback.

Requires:
    pip install ultralytics opencv-python numpy

Usage (Terminal/PowerShell):
    python count_pills_yolo.py
    # หรือกำหนดเอง:
    python count_pills_yolo.py --source "C:\path\to\folder" --show
"""

from pathlib import Path
import argparse
import numpy as np
import cv2
from ultralytics import YOLO

# ---------- CONFIG ----------
SOURCE_DEFAULT = r"C:\Users\LOBSTER69\Documents\WORK\Windows\CODE_OTHER\0005 - Count_the_medicine\DATA_SET"
MODEL_DEFAULT = "yolov8n.pt"  # YOLO nano (เล็กสุด)

# ---------- HELPERS ----------
def shape_filter(box, img_w, img_h, w_hrange=(0.80, 1.20), area_range=(0.00015, 0.06)):
    x1, y1, x2, y2 = box
    w, h = max(0, x2 - x1), max(0, y2 - y1)
    if w == 0 or h == 0:
        return False
    aspect = w / h
    area_ratio = (w * h) / (img_w * img_h)
    return (w_hrange[0] <= aspect <= w_hrange[1]) and (area_range[0] <= area_ratio <= area_range[1])

def annotate_and_count(image_bgr, detections, title="Pill count"):
    img = image_bgr.copy()
    for (x1, y1, x2, y2, conf) in detections:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, f"pill {conf:.2f}", (x1, max(0, y1 - 6)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(img, f"{title}: {len(detections)}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)
    return img

# ---------- CORE ----------
def count_pills_yolo(source,
                     model_path=MODEL_DEFAULT,
                     conf=0.05, iou=0.5, imgsz=640,
                     use_fallback=True, show=False, save=True):
    model = YOLO(model_path)

    def process_frame(frame_bgr):
        H, W = frame_bgr.shape[:2]
        results = model.predict(
            source=frame_bgr, conf=conf, iou=iou, imgsz=imgsz,
            agnostic_nms=True, max_det=1000, verbose=False
        )

        accepted = []
        for r in results:
            if r.boxes is None:
                continue
            for b in r.boxes:
                x1, y1, x2, y2 = b.xyxy[0].cpu().numpy()
                confv = float(b.conf)
                box = (int(x1), int(y1), int(x2), int(y2))
                if shape_filter(box, W, H):
                    accepted.append((*box, confv))

        # Fallback ถ้า YOLO ไม่เจอ
        if use_fallback and len(accepted) == 0:
            gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 5)
            circles = cv2.HoughCircles(
                gray, cv2.HOUGH_GRADIENT, dp=1.2,
                minDist=max(8, min(W, H) // 20),
                param1=80, param2=25,
                minRadius=max(6, min(W, H) // 100),
                maxRadius=min(W, H) // 8
            )
            if circles is not None:
                for (cx, cy, r) in np.round(circles[0]).astype(int):
                    x1, y1, x2, y2 = cx - r, cy - r, cx + r, cy + r
                    accepted.append((x1, y1, x2, y2, 0.0))

        annotated = annotate_and_count(frame_bgr, accepted, title="Pill count")
        return annotated, len(accepted)

    src_path = str(source)
    is_camera = isinstance(source, int) or (isinstance(source, str) and source.isdigit())
    is_video = (not is_camera) and Path(src_path).suffix.lower() in (".mp4", ".mov", ".avi", ".mkv", ".m4v")

    if is_camera or is_video:
        cap = cv2.VideoCapture(0 if is_camera else src_path)
        if not cap.isOpened():
            raise RuntimeError(f"Cannot open source: {source}")
        writer = None
        out_paths, latest_count = [], 0
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")

        while True:
            ok, frame = cap.read()
            if not ok:
                break
            annotated, n = process_frame(frame)
            latest_count = n

            if show:
                cv2.imshow("Pill counter", annotated)
                if cv2.waitKey(1) & 0xFF == 27:
                    break

            if save and writer is None:
                out_path = Path("pill_count_output.mp4")
                writer = cv2.VideoWriter(str(out_path), fourcc, 20.0,
                                         (annotated.shape[1], annotated.shape[0]))
                out_paths.append(str(out_path))
            if save and writer is not None:
                writer.write(annotated)

        if writer is not None:
            writer.release()
        cap.release()
        if show:
            cv2.destroyAllWindows()
        print(f"[Video/Camera] Latest frame count = {latest_count}")
        return latest_count, out_paths

    p = Path(src_path)
    if not p.exists():
        raise FileNotFoundError(f"Source not found: {src_path}")

    if p.is_dir():
        images = []
        for ext in ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff"):
            images += list(p.glob(ext))
        images.sort()
    else:
        images = [p]

    saved_paths, last_count = [], 0
    for img_path in images:
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"[WARN] Cannot read: {img_path}")
            continue
        annotated, n = process_frame(img)
        last_count = n
        if save:
            out_path = img_path.with_name(img_path.stem + "_count.jpg")
            cv2.imwrite(str(out_path), annotated)
            saved_paths.append(str(out_path))
            print(f"[Image] {img_path.name}: count = {n} -> saved {out_path.name}")

        if show:
            cv2.imshow("Pill counter", annotated)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

    return last_count, saved_paths

# ---------- MAIN ----------
def main():
    ap = argparse.ArgumentParser(description="Count pills with YOLOv8n + shape filter + HoughCircles fallback")
    ap.add_argument("--source", default=SOURCE_DEFAULT,
                    help="รูป/โฟลเดอร์/วิดีโอ หรือ 0 สำหรับกล้อง (default = โฟลเดอร์ของคุณ)")
    ap.add_argument("--model", default=MODEL_DEFAULT, help="โมเดล YOLO (default: yolov8n.pt)")
    ap.add_argument("--conf", type=float, default=0.05, help="confidence threshold")
    ap.add_argument("--iou", type=float, default=0.5, help="IoU threshold")
    ap.add_argument("--imgsz", type=int, default=640, help="image size for inference")
    ap.add_argument("--show", action="store_true", help="แสดงหน้าต่างผลลัพธ์")
    ap.add_argument("--no-fallback", dest="use_fallback", action="store_false",
                    help="ปิด HoughCircles fallback")

    # สำคัญ: รองรับ Jupyter โดยเมิน args แปลก ๆ เช่น --f=...
    args, _unknown = ap.parse_known_args()

    count, outputs = count_pills_yolo(
        source=args.source,
        model_path=args.model,
        conf=args.conf,
        iou=args.iou,
        imgsz=args.imgsz,
        use_fallback=args.use_fallback,
        show=args.show,
        save=True
    )

    print(f"\nFinal count (last processed item) = {count}")
    if outputs:
        print("Saved outputs:")
        for p in outputs:
            print("  -", p)

if __name__ == "__main__":
    main()
